In [2]:
!pip3 install -Uqq fastai

     |████████████████████████████████| 194kB 21.6MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 


In [1]:
import fastai
fastai.__version__

'2.4'

In [2]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [3]:
from fastai.text.all import *
import pandas as pd
import numpy as np

In [4]:
path = Path('/content/gdrive/MyDrive/subcellular-location/v2')

In [5]:
Path.BASE_PATH = path

In [6]:
path.ls()

(#5) [Path('LM_data_2021-03-11.csv'),Path('proteinClassifier.ipynb'),Path('data'),Path('models'),Path('AA_LM_model.pk')]

In [7]:
data = path / Path('data/raw/LM_data_2021-03-11.csv')

In [8]:
df = pd.read_csv(data, sep=';')
df.head()

,Entry,Entry name,Sequence
0,P68307,NU3M_BALMU,MNLLLTLLTNTTLALLLVFIAFWLPQLNVYAEKTSPYECGFDPMGSARLPFSMKFFLVAITFLLFDLEIALLLPLPWAIQSNNLNTMLTMALFLISLLAASLAYEWTQEGLEWAE
1,P0CY61,O162_CONBU,MKLTCVLIIAVLFLTAITADDSRDKQVYRAVGLIDKMRRIRASEGCRKKGDRCGTHLCCPGLRCGSGRAGGACRPPYN
2,Q0VIL3,OTOMP_DANRE,MDLPGGHLAVVLFLFVLVSMSTENNIIRWCTVSDAEDQKCLDLAGNATARNLRGQLVCVRGQSPTDCMKQIKNGTADASTMYADEIYTAGFCYGLDVAVGESYNGVDGINYYVVALARTSSSDLSLLEMHERSSCHPGMRTTVGWTVPIGFLVNTSQISVDVQCNFPHAVGDFFGYSCVPGVKDPEHDPKGNNPRNLCEACIGDENDRHICANNPRERHFGEAGALRCVAENLGDVAFVKHTTVFDNMQGKNQESWALDLELEDLKLLCPDGSEANLFQHESCHLAVVPTNAVVVRLEDKCRVYKFLERVQNAFGNTTEGFSLFSSVNYGQPDVLFSDSTKKLLRVMGTYTSWLGPSYTTILRAFECEGLC
3,A1W9I4,NUSB_ACISJ,MTDSTHPTPSARPPRQPRTGTTGTGARKAGSKSGRSRAREFALQALYQHLVGGNDATAIDVFTRDLSGFHKADAAHYDALLHGCITTAQYMDELIAPQLDRKMSEISPIEHAVMWIGVYEFQHCQDVPWRVVINECIELAKEFGGTDGHKYVNGVLNGLAPQLRATEVAADKAAARG
4,Q8DBX0,OMPU_VIBVU,MKKTLIALSVSAAAVATGVNAAELYNQDGTSLDMGGRAEARLSMKDGKVADNSRIRLNFLGKVEIQDGLYGVGFYEGEFTTADNADGSDLDNRYTYAGLGGKFGEVTYGKNDGALGVITDFTDIMAYHGNSAADKIAVADRVDNMMSYKGQFDALSVKASYRFADRADSSKNNVDNTYIDNGKDGYSLSAIYAIGQTGLTLGGGYADQDKSNEYMLAASYTMGDLYFAGVFTDGEKDYGTNGDYSHRGFSSVEDYTGYELAAKYTMGQTVFTTTYNNAETDGYTSTDNFAVDATYYFKPNFRGYVSYNFNLLDAGDKIGTSTISKADAEDELALGLRYDF


In [9]:
mini_data = df[:1000]
mini_data.shape

(1000, 3)

In [10]:
def create_vocab(df, protein_seqs_column, kmer_sz, stride=1, eos_token=True, pad_token=False):
    kmers = set()
        
    # Map kmers for one-hot encoding
    kmer_to_id = dict()
    id_to_kmer = dict()

    # Loop over the protein sequences
    for protein_seq in df[protein_seqs_column]:
        # Loop over the sequence and add the amino acid if it is not in kmers set.
        seq_len = len(protein_seq)


        for i in range(0, seq_len - (kmer_sz - 1), stride):

            kmer = protein_seq[i: i + kmer_sz]

            if kmer not in list(kmers):
                ind = len(kmers)
                kmers.add(kmer)

                # Also create the dictionary
                kmer_to_id[kmer] = ind
                id_to_kmer[ind] = kmer

    if eos_token:
        token = '<EOS>'
        ind = len(kmers)
        
        kmers.add(token)

        # Also create the dictionary
        kmer_to_id[token] = ind
        id_to_kmer[ind] = token

    if pad_token:
        token = '<PAD>'
        ind = len(kmers)
        
        kmers.add(token)

        # Also create the dictionary
        kmer_to_id[token] = ind
        id_to_kmer[ind] = token

    vocab_sz = len(kmers)

    assert vocab_sz == len(kmer_to_id.keys())
    
    return kmer_to_id, id_to_kmer, vocab_sz

In [11]:
def tokenize(df, protein_seqs_column, kmer_sz, stride=1, eos_token=True, premade_vocab=False, pad_token = False):
    
    
    # Create the vocabulary
    if not premade_vocab:
        
        kmer_to_id, id_to_kmer, vocab_sz = create_vocab(df, protein_seqs_column, kmer_sz, stride, eos_token, pad_token=pad_token)
                
    else:
        kmer_to_id, id_to_kmer = premade_vocab
        vocab_sz = len(kmer_to_id.keys())
            
    # Tokenize the sequences in the DF

    tokenized = []
    for i, protein_seq in enumerate(df[protein_seqs_column], 0):
        sequence = []
        
        # If the kmer can't be found these indexes should be deleted
        remove_idxs = []
        
        # Loop over the protein sequence
        for i in  range(len(protein_seq) - (kmer_sz -1)):
            # Convert kmer to integer
            kmer = protein_seq[i: i + kmer_sz]
            
            # For some reason, some kmers miss. Thus these sequences have to be removed
            try:
                sequence.append(kmer_to_id[kmer])
            except:
                remove_idxs.append(i)
                
        if eos_token:
            sequence.append(kmer_to_id['<EOS>'])
            
        tokenized.append(Tensor(sequence))
            
    df['tokenized_seqs'] = tokenized
    
    df.drop(remove_idxs, inplace=True)
    
    return df, vocab_sz, kmer_to_id, id_to_kmer

In [12]:
KMER_SIZE = 1

In [13]:
df, vocab_sz, kmer_to_id, id_to_kmer = tokenize(mini_data, 'Sequence', KMER_SIZE, pad_token=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
print(kmer_to_id)

{'M': 0, 'N': 1, 'L': 2, 'T': 3, 'A': 4, 'V': 5, 'F': 6, 'I': 7, 'W': 8, 'P': 9, 'Q': 10, 'Y': 11, 'E': 12, 'K': 13, 'S': 14, 'C': 15, 'G': 16, 'D': 17, 'R': 18, 'H': 19, 'X': 20, '<EOS>': 21, '<PAD>': 22}


In [15]:
df['tokenized_seqs'].head()

0    [tensor(0.), tensor(1.), tensor(2.), tensor(2.), tensor(2.), tensor(3.), tensor(2.), tensor(2.), tensor(3.), tensor(1.), tensor(3.), tensor(3.), tensor(2.), tensor(4.), tensor(2.), tensor(2.), tensor(2.), tensor(5.), tensor(6.), tensor(7.), tensor(4.), tensor(6.), tensor(8.), tensor(2.), tensor(9.), tensor(10.), tensor(2.), tensor(1.), tensor(5.), tensor(11.), tensor(4.), tensor(12.), tensor(13.), tensor(3.), tensor(14.), tensor(9.), tensor(11.), tensor(12.), tensor(15.), tensor(16.), tensor(6.), tensor(17.), tensor(9.), tensor(0.), tensor(16.), tensor(14.), tensor(4.), tensor(18.), tensor...
1    [tensor(0.), tensor(13.), tensor(2.), tensor(3.), tensor(15.), tensor(5.), tensor(2.), tensor(7.), tensor(7.), tensor(4.), tensor(5.), tensor(2.), tensor(6.), tensor(2.), tensor(3.), tensor(4.), tensor(7.), tensor(3.), tensor(4.), tensor(17.), tensor(17.), tensor(14.), tensor(18.), tensor(17.), tensor(13.), tensor(10.), tensor(5.), tensor(11.), tensor(18.), tensor(4.), tensor(5.), tensor

In [16]:
def add_padding(x, pad):
  pad_token = [22]
  padding = Tensor((pad - len(x)) * pad_token)
  conc = torch.cat([x, padding], dim=0)

  return conc

In [17]:
test_seq = df['tokenized_seqs'][0]
len(test_seq)

116

In [18]:
len(add_padding(test_seq, 130))

130

In [19]:
add_padding(test_seq, 130)

tensor([ 0.,  1.,  2.,  2.,  2.,  3.,  2.,  2.,  3.,  1.,  3.,  3.,  2.,  4.,
         2.,  2.,  2.,  5.,  6.,  7.,  4.,  6.,  8.,  2.,  9., 10.,  2.,  1.,
         5., 11.,  4., 12., 13.,  3., 14.,  9., 11., 12., 15., 16.,  6., 17.,
         9.,  0., 16., 14.,  4., 18.,  2.,  9.,  6., 14.,  0., 13.,  6.,  6.,
         2.,  5.,  4.,  7.,  3.,  6.,  2.,  2.,  6., 17.,  2., 12.,  7.,  4.,
         2.,  2.,  2.,  9.,  2.,  9.,  8.,  4.,  7., 10., 14.,  1.,  1.,  2.,
         1.,  3.,  0.,  2.,  3.,  0.,  4.,  2.,  6.,  2.,  7., 14.,  2.,  2.,
         4.,  4., 14.,  2.,  4., 11., 12.,  8.,  3., 10., 12., 16.,  2., 12.,
         8.,  4., 12., 21., 22., 22., 22., 22., 22., 22., 22., 22., 22., 22.,
        22., 22., 22., 22.])

In [20]:
df = df.sort_values(by='tokenized_seqs', key = lambda x: x.str.len())

In [21]:
# Hoe kan ik de average van iedere locatie bepalen...
# Ik ga aan alle sequenties padding toevoegen, en dan voor alle plekken een softmax probability uitrekenen

In [22]:
max_seq_length = max(df['tokenized_seqs'].str.len())

In [23]:
df['tokenized_seqs'] = df['tokenized_seqs'].map(lambda x: add_padding(x, max_seq_length))

In [24]:
df['tokenized_seqs'].str.len()

348    3342
741    3342
738    3342
734    3342
168    3342
       ... 
45     3342
409    3342
414    3342
376    3342
384    3342
Name: tokenized_seqs, Length: 1000, dtype: int64

In [25]:
data = tuple(df['tokenized_seqs'])
stacked = torch.stack(data)
stacked.shape

torch.Size([1000, 3342])

In [26]:
def one_hot_enc(x, vocab_sz):
  lst = list(x)
  one_hot = torch.eye(vocab_sz)[lst]
  return one_hot

In [27]:
one_hot_stacked = []

In [28]:
for _, x in enumerate(stacked):
  one_hot = one_hot_enc(x, vocab_sz)
  one_hot_stacked.append(one_hot)

one_hot_stacked = torch.stack(one_hot_stacked)

In [29]:
one_hot_stacked.shape

torch.Size([1000, 3342, 23])

In [30]:
one_hot_stacked.sum(dim=2).shape

torch.Size([1000, 3342])

In [31]:
def softmax_prob(matrix):
  return np.exp(matrix.sum(dim=0)) / np.exp(matrix.sum(dim=0)).sum(axis=0)

In [42]:
average = softmax_prob(one_hot_stacked)
average.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


torch.Size([3342, 23])

In [43]:
average = torch.nan_to_num(test, nan=0.90)

In [62]:
# Accuracy
total = 0
right = 0

# Loop over every input
for i, x in enumerate(one_hot_stacked):
  # Loop over every one hot encoding but dont include padding that will make it right to often
  for j, k in enumerate(x):
    if torch.argmax(average[j]) == torch.argmax(k) and torch.argmax(k) != 22:
      right += 1
    total += 1

accuracy = right / total * 100
accuracy

0.466457211250748

Met padding mee geteld, maar dat geeft scheve uitslag omdat bijna alles met heel veel padding is.

In [63]:
# Accuracy
total = 0
right = 0

# Loop over every input
for i, x in enumerate(one_hot_stacked):
  # Loop over every one hot encoding but dont include padding that will make it right to often
  for j, k in enumerate(x):
    if torch.argmax(average[j]) == torch.argmax(k):
      right += 1
    total += 1

accuracy = right / total * 100
accuracy

89.14144225014962